In [2]:
import pandas as pd
import datetime

This notebook is preliminary ETL for data from the weather API. <br>
We work with data for Union Square in San Francisco; (37.7879, -122.4079) for API.  <br>
We pulled data from 2000-01-01 to 2020-12-31. <br>
We took temperature, humidity, rainfal, snowfal, cloudcover, windspeed, and wind-direction hourly. <br>
<hr> 
We will also take max temp, min temp, rain, snow, and percip-hours daily. <br>
These will need separate and different dataframe processing. <br>
We are taking in data via downloaded csv, and we need separate daily/hourly csv files because different column headers, numbers of columns.<br>
<hr>
We chose ISO format for date/time; and USian (not-metric) units for the rest.<br>
We chose a random time zone; in the future, we will need to specify the timezone appropriate to the location.

In [3]:
# Load the dataset.
# rename columns because provided column headers contain non-ascii characters
file_path ="era5wL_hourly.csv"
columns = ['time',
           'temperature_2m_degF', 
           'relativehumidity_2m_perc', 
           'rain_inch',
           'snowfall_cm', 
           'cloudcover_perc', 
           'windspeed_10m_mph',
           'winddirection_10m_deg']

raw_hrly_df1 = pd.read_csv(file_path, skiprows=4, names=columns)
raw_hrly_df1.head()

,time,temperature_2m_degF,relativehumidity_2m_perc,rain_inch,snowfall_cm,cloudcover_perc,windspeed_10m_mph,winddirection_10m_deg
0,2000-01-01T00:00,45.6,92,0.0,0.0,29,8.8,306.0
1,2000-01-01T01:00,45.0,94,0.0,0.0,29,8.4,316.0
2,2000-01-01T02:00,43.1,98,0.0,0.0,71,8.1,328.0
3,2000-01-01T03:00,44.9,96,0.0,0.0,66,5.9,335.0
4,2000-01-01T04:00,44.7,96,0.0,0.0,72,5.7,339.0


When wind speed is 0, wind direction is NaN. Otherwise, wind directions varies from 1 to 360.<br>
We replace NaNs with 0s to avoid errors; and losing no data as 0 never appears in the original.

In [4]:
raw_hrly_df1["winddirection_10m_deg"].fillna(0, inplace=True)

To parse date-time, we first convert it from ISO format to python's datetime; and then extract three numeric fields year, day_of_year, and hour. <br>
<i> actually, we should do only get 'hour' here; get others and also 'month' after merging with the daily dataframe - no sense doing it 24 times for each day.</i><br>
The day_of_year field runs from 1 to 365 (366 for leap years); for example, february 3 would be 34. We'll want this for comparing same days in different years, especially for machine learning. <br>
To evaluate time intervals in user requests, we'll use the full date-time.

In [5]:
raw_hrly_df1["date_time"] = raw_hrly_df1['time'].map(lambda x: datetime.datetime.fromisoformat(x))

In [6]:
raw_hrly_df1['year'] = raw_hrly_df1["date_time"].map(lambda x: x.year)

In [7]:
raw_hrly_df1['hour'] = raw_hrly_df1["date_time"].map(lambda x: x.hour)

In [9]:
# this code stolen from
# https://www.w3resource.com/python-exercises/date-time-exercise/python-date-time-exercise-11.php
# which says nothing about licenses
def day_of_year(x):
    return (x - datetime.datetime(x.year, 1, 1)).days + 1

In [10]:
raw_hrly_df1['day_of_year'] = raw_hrly_df1["date_time"].map(lambda x: day_of_year(x))

In [11]:
raw_hrly_df1.describe()

,temperature_2m_degF,relativehumidity_2m_perc,rain_inch,snowfall_cm,cloudcover_perc,windspeed_10m_mph,winddirection_10m_deg,year,hour,day_of_year
count,184104.000000,184104.000000,184104.000000,184104.0,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000
mean,55.518150,80.822997,0.002685,0.0,36.980022,8.516822,243.822323,2010.000000,11.500000,183.143136
std,5.569403,12.769135,0.014148,0.0,36.118408,4.573279,81.371893,6.055963,6.922205,105.449278
min,35.800000,16.000000,0.000000,0.0,0.000000,0.000000,0.000000,2000.000000,0.000000,1.000000
25%,52.000000,74.000000,0.000000,0.0,2.000000,5.000000,211.000000,2005.000000,5.750000,92.000000
50%,55.300000,83.000000,0.000000,0.0,27.000000,7.800000,277.000000,2010.000000,11.500000,183.000000
75%,59.000000,91.000000,0.000000,0.0,68.000000,11.400000,297.000000,2015.000000,17.250000,274.000000
max,88.800000,100.000000,0.370000,0.0,100.000000,35.400000,360.000000,2020.000000,23.000000,366.000000


## aaaaa, this is not the same as what Laurina was getting! 
her outputs below.

In [69]:
raw_hrly_df1.describe()

,temperature_2m_degF,relativehumidity_2m_perc,rain_inch,snowfall_cm,cloudcover_perc,windspeed_10m_mph,winddirection_10m_deg
count,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000
mean,62.685529,60.031107,0.002539,0.000006,24.462673,5.317476,214.860362
std,13.893239,22.489070,0.013077,0.000937,32.715640,3.088230,77.533407
min,24.200000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,52.700000,42.000000,0.000000,0.000000,0.000000,3.100000,180.000000
50%,60.900000,62.000000,0.000000,0.000000,6.000000,4.800000,211.000000
75%,71.400000,80.000000,0.000000,0.000000,36.000000,7.000000,250.000000
max,110.600000,100.000000,0.437000,0.210000,100.000000,31.100000,360.000000


In [70]:
raw_hrly_df1["time"]

0         2000-01-01T00:00
1         2000-01-01T01:00
2         2000-01-01T02:00
3         2000-01-01T03:00
4         2000-01-01T04:00
                ...       
184099    2020-12-31T19:00
184100    2020-12-31T20:00
184101    2020-12-31T21:00
184102    2020-12-31T22:00
184103    2020-12-31T23:00
Name: time, Length: 184104, dtype: object

In [107]:
raw_hrly_df1.head()

,time,temperature_2m_degF,relativehumidity_2m_perc,rain_inch,snowfall_cm,cloudcover_perc,windspeed_10m_mph,winddirection_10m_deg,month,day,hour,time_new
0,2000-01-01T00:00,40.8,95,0.0,0.0,35,4.5,147.0,1,1,0,2000-01-01 00:00:00
1,2000-01-01T01:00,40.5,95,0.0,0.0,42,5.1,146.0,1,1,1,2000-01-01 01:00:00
2,2000-01-01T02:00,38.4,99,0.0,0.0,46,2.2,143.0,1,1,2,2000-01-01 02:00:00
3,2000-01-01T03:00,42.2,94,0.0,0.0,63,2.5,117.0,1,1,3,2000-01-01 03:00:00
4,2000-01-01T04:00,41.5,93,0.0,0.0,79,2.5,80.0,1,1,4,2000-01-01 04:00:00


In [112]:
raw_hrly_df1.describe()

,temperature_2m_degF,relativehumidity_2m_perc,rain_inch,snowfall_cm,cloudcover_perc,windspeed_10m_mph,winddirection_10m_deg,month,day,hour,day_of_year,hour_of_day
count,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000,184104.000000
mean,62.685529,60.031107,0.002539,0.000006,24.462673,5.317476,214.860362,6.522487,15.730935,11.500000,183.143136,11.500000
std,13.893239,22.489070,0.013077,0.000937,32.715640,3.088230,77.533407,3.448834,8.800664,6.922205,105.449278,6.922205
min,24.200000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,52.700000,42.000000,0.000000,0.000000,0.000000,3.100000,180.000000,4.000000,8.000000,5.750000,92.000000,5.750000
50%,60.900000,62.000000,0.000000,0.000000,6.000000,4.800000,211.000000,7.000000,16.000000,11.500000,183.000000,11.500000
75%,71.400000,80.000000,0.000000,0.000000,36.000000,7.000000,250.000000,10.000000,23.000000,17.250000,274.000000,17.250000
max,110.600000,100.000000,0.437000,0.210000,100.000000,31.100000,360.000000,12.000000,31.000000,23.000000,366.000000,23.000000


### Next steps.
<ul>
    <li> Unpivot data on hour to get one row for each day with 6x24 = 144 weather-at-hour columns. Keep/add a 'date' column in datetime format, matching what the daily data looks like.</li>
    <li> Read in daily data, convert dates to datetime, add 'year' and 'day_of_year' columns. </li>
    <li> Merge the two dataframes on *date*. </li>
</ul>

For machine learning of seasons, we might want to unpivot on year, to have 366 rows with 20x the number of columns?<br>
For machine learning of month-labeling, we'll also want a 'month' column.<br>
If joining the two dataframes is slow, maybe convert the date joining field to unix format?
      